<font color="grey">
    
### Before running the code, a .env file must be created to store the OpenAI API key.

- In the .env file, add the following line: OPENAI_API_KEY=*your openai api key*
- More to read: https://bhavikjikadara.medium.com/chroma-langchain-transforming-document-q-a-5efdf0e11c4b

</font>

In [21]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
load_dotenv()

True

<font color="grey">
    
- When *load_dotenv()* returns True, it indicates that the OpenAI API key has been set correctly.
</font>

In [22]:
import pandas as pd
books=pd.read_csv(r'F:\learning\LLM\data\books_cleaned.csv')
books["tagged_description"]=books["tagged_description"].apply(lambda x: x.replace('"',''))
books["tagged_description"].to_csv(r"F:\learning\LLMtagged_description.txt", sep = "\n", index = False, header = False)

<font color="grey">
    
- Here we only consider the text description of each book.
</font>

<font color="grey">
    
CharacterTextSplitter will only split on separator (which is '\n\n' by default). chunk_size is the maximum chunk size that will be split if splitting is possible. If a string starts with n characters, has a separator, and has m more characters before the next separator then the first chunk size will be n if chunk_size < n + m + len(separator).
                                                                                                                                                                                                                                                                                                                        
</font>                                                                                                                                                                                                                                                                                                                        

In [23]:
raw_documents = TextLoader(r"F:\learning\LLMtagged_description.txt", encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1168, which is longer than the specified 1
Created a chunk of size 1214, which is longer than the specified 1
Created a chunk of size 373, which is longer than the specified 1
Created a chunk of size 309, which is longer than the specified 1
Created a chunk of size 477, which is longer than the specified 1
Created a chunk of size 482, which is longer than the specified 1
Created a chunk of size 960, which is longer than the specified 1
Created a chunk of size 188, which is longer than the specified 1
Created a chunk of size 843, which is longer than the specified 1
Created a chunk of size 274, which is longer than the specified 1
Created a chunk of size 191, which is longer than the specified 1
Created a chunk of size 875, which is longer than the specified 1
Created a chunk of size 1088, which is longer than the specified 1
Created a chunk of size 1189, which is longer than the specified 1
Created a chunk of size 304, which is longer than the specified 1
Create

In [24]:
# Check if our download is correct. If k=[], then it is correct
k=[]
for i in range(len(documents)):
    if documents[i].page_content[:13]==str(books.loc[i,'isbn13']): # 13 is the length of isbn13 number
        pass
    else: k.append(i)
if k:
    print("The loading is not correct")
else: print("The loading is correct")

The loading is correct


In [25]:
# The text description for our first book
documents[0]

Document(metadata={'source': 'F:\\learning\\LLMtagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of l

In [26]:
# OpenAI will charge a small fee for running this code. Save the Chroma data (if it's your first time run the code)
persist_directory=r'F:\learning\LLM\db_books_test'
embeddings = OpenAIEmbeddings()

# If it's your 1st time to run the code, use this:
#db_books = Chroma.from_documents(documents, embeddings , persist_directory=persist_directory) #it is needed if the Chroma data is not saved

# If it's not your 1st time to run the code and you have saved the data, use this:
# I've already run the code above, so now I can simply import it and use it directly. 
db_books = Chroma(persist_directory=persist_directory,embedding_function=embeddings)

<font color="grey">
Now we can use it for similarity search. For example, if the input query is *"My best friend is very  nice"*, we can get 2 recommend books:
</font>

In [27]:
# Now you can use it for similarity search
results = db_books.similarity_search("My best friend is very  nice", k=2)
for doc in results:
    print(doc.page_content)

9780786808717 A very special puddle sets Violet the mouse off on her latest nature discovery. It is through this puddle that Violet observes the effect rain has on the world around her. A Mylar puddle on the last page offers children a chance to see their reflection in a puddle, just like Violet!
9780763620875 When Judy Moody gets serious about protecting the environment, her little brother Stink thinks she is overdoing it, but she manages to inspire her third grade class to undertake an award-winning, environment-saving project. Reprint.


<font color="grey">
Another example: Recommend 10 books similar to the query, but return the book information instead of the text description as shown above.
</font>

In [28]:
query="A book to teach children about nature."
docs=db_books.similarity_search(query,k=10)
#the detail of the first recommend book
books[books['isbn13']==int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description
3747,9780786808069,0786808063,Baby Einstein: Neighborhood Animals,Marilyn Singer;Julie Aigner-Clark,Juvenile Fiction,http://books.google.com/books/content?id=X9a4P...,Children will discover the exciting world of t...,2001.0,3.89,16.0,180.0,Baby Einstein: Neighborhood Animals,9780786808069 Children will discover the excit...


In [29]:
# The whole recommendation
docs

[Document(id='299a51ce-b107-4257-840c-6e92483d3106', metadata={'source': 'tagged_description.txt'}, page_content='9780786808069 Children will discover the exciting world of their own backyard in this introduction to familiar animals from cats and dogs to bugs and frogs. The combination of photographs, illustrations, and fun facts make this an accessible and delightful learning experience.'),
 Document(id='0793826e-57b3-4244-8f40-18159cd39015', metadata={'source': 'tagged_description.txt'}, page_content="9780786808380 Introduce your babies to birds, cats, dogs, and babies through fine art, illustration, and photographs. These books are a rare opportunity to expose little ones to a range of images on a single subject, from simple child's drawings and abstract art to playful photos. A brief text accompanies each image, introducing the baby to some basic -- and sometimes playful -- information about the subjects."),
 Document(id='f1dd8b2c-7906-4871-bc39-e080db8d1fdc', metadata={'source': '

In [30]:
# Write a function to recommend 10 books similar to query.
def retrieve_semantic_recommendations(query:str,top_k:int=10)->pd.DataFrame:
    recs=db_books.similarity_search(query,k=15)
    books_list=[]
    for i in range(len(recs)):
        books_list+=[int(recs[i].page_content.strip('"').split()[0])]#isbn number
    return books[books['isbn13'].isin(books_list)].head(top_k)
query="A book to teach children about nature."
books_recommend=retrieve_semantic_recommendations(query)
books_recommend.to_csv(r"F:\learning\LLM\step2_recommended_books.csv", index = False)

<font color="grey">
    
### The whole code
</font>

In [31]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
books=pd.read_csv(r'F:\learning\LLM\data\books_cleaned.csv')
books["tagged_description"]=books["tagged_description"].apply(lambda x: x.replace('"',''))
books["tagged_description"].to_csv(r"F:\learning\LLMtagged_description.txt", sep = "\n", index = False, header = False)

raw_documents = TextLoader(r"F:\learning\LLMtagged_description.txt", encoding='utf-8').load()
text_splitter = CharacterTextSplitter(chunk_size=1, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

##############################################################################################
# OpenAI will charge a small fee for running this code. Save the Chroma data (if it's your first time run the code)
persist_directory=r'F:\learning\LLM\db_books_test'
embeddings = OpenAIEmbeddings()

# If it's your 1st time to run the code, use this:
#db_books = Chroma.from_documents(documents, embeddings , persist_directory=persist_directory) #it is needed if the Chroma data is not saved

# If it's not your 1st time to run the code and you have saved the data, use this:
# I've already run the code above, so now I can simply import it and use it directly. 
db_books = Chroma(persist_directory=persist_directory,embedding_function=embeddings)
##############################################################################################

# Write a function to recommend 10 books similar to query.
def retrieve_semantic_recommendations(query:str,top_k:int=10)->pd.DataFrame:
    recs=db_books.similarity_search(query,k=15)
    books_list=[]
    for i in range(len(recs)):
        books_list+=[int(recs[i].page_content.strip('"').split()[0])]#isbn number
    return books[books['isbn13'].isin(books_list)].head(top_k)
query="A book to teach children about nature."
books_recommend=retrieve_semantic_recommendations(query)
books_recommend.to_csv(r"F:\learning\LLM\step2_recommended_books.csv", index = False)

Created a chunk of size 1168, which is longer than the specified 1
Created a chunk of size 1214, which is longer than the specified 1
Created a chunk of size 373, which is longer than the specified 1
Created a chunk of size 309, which is longer than the specified 1
Created a chunk of size 477, which is longer than the specified 1
Created a chunk of size 482, which is longer than the specified 1
Created a chunk of size 960, which is longer than the specified 1
Created a chunk of size 188, which is longer than the specified 1
Created a chunk of size 843, which is longer than the specified 1
Created a chunk of size 274, which is longer than the specified 1
Created a chunk of size 191, which is longer than the specified 1
Created a chunk of size 875, which is longer than the specified 1
Created a chunk of size 1088, which is longer than the specified 1
Created a chunk of size 1189, which is longer than the specified 1
Created a chunk of size 304, which is longer than the specified 1
Create